In [2]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from prefixed import Float

def findCross(signal: np.ndarray, triggerLevel: float, hysteresis: float):
  upValue = triggerLevel + hysteresis
  downValue = triggerLevel - hysteresis
  isUp = signal[0] > upValue
  cross = np.array([], dtype=int)
  for i, v in enumerate(signal):
    if isUp and v < downValue:
      isUp = False
      cross = np.append(cross, [i])
    elif not isUp and v > upValue:
      isUp = True

  return cross

def findWave(signal: np.ndarray, nWaves: int, triggerLevel: float):
  """ Finds trigger level crossing with hysteresis """
  hysteresis = 0.3

  cross = findCross(signal, triggerLevel, hysteresis)
  print(cross)
  # crossDiff = np.diff(cross)

  # print(np.mean(crossDiff))
  # crossDown = np.flatnonzero(upMask2)
  # print(crossDown)
  # cross = np.flatnonzero(upMask1 | upMask2)

  # """ if few crossing points are found ignore trigger """
  # if cross.size < 5:
  #   return np.array([]), []

  # """ Select a center crossing point """
  # centerCross = cross.size - (nWaves * 2) - 1
  # if signal[cross[centerCross]] < signal[cross[centerCross] + 10]:
  #   centerCross -= 1

  # triggered = np.copy(signal[cross[centerCross - 1]:cross[centerCross + (2 * nWaves)]])

  # refCross = cross[centerCross - 1: centerCross + 2] - cross[centerCross - 1]
  
  return cross



# df = pd.read_csv('test.csv')
# data = np.array(df.values).transpose()[2]
fs = 10e3
f1 = 16
f2 = 1000

data = 2 * np.sin(np.array(range(1000)) * 2 * np.pi * f1 / fs) + 0.2 * np.sin(np.array(range(1000)) * 2 * np.pi * f2 / fs)

xAxis = np.array(range(data.size))

cross = findWave(data, 1, 0.5)

fig1 = make_subplots(specs=[[{"secondary_y": True}]])
fig1.add_trace(go.Scatter(x=xAxis, y=data, name='TestData'))
# fig1.add_trace(go.Scatter(x=[xAxis[i] for i in up], y=[data[i] for i in up], mode='markers'))
# fig1.add_trace(go.Scatter(x=[xAxis[i] for i in down], y=[data[i] for i in down], mode='markers'))
fig1.add_trace(go.Scatter(x=[xAxis[i] for i in cross], y=[data[i] for i in cross], mode='markers'))

fig1.show()





[297 918]


In [26]:


def getSpectrum(data :np.ndarray, fs):
  N = data.size
  Y = np.fft.fft(data)
  Xf = np.fft.fftfreq(N, 1/fs)[:N//2]

  Yf = 2.0/N * np.abs(Y[0:(N//2)])[1:]
  phase = np.angle(Y)[:N//2] / np.pi

  return Xf[1:], Yf, phase



fs = 1000

wave = 5 * np.sin(np.arange(1000) * 2 * np.pi * 60 / fs) + 2 * np.sin(np.arange(1000) * 2 * np.pi * 150 / fs) + 3

xf, yf, phase = getSpectrum(wave, fs)

idx = np.argmax(yf)

fig2 = make_subplots(specs=[[{"secondary_y": True}]])
fig2.add_trace(go.Scatter(x=xf, y=yf, name='power'))
fig2.add_trace(go.Scatter(x=xf, y=phase, name='fase'))

fig2.show()